# Avoiding Road Blocks Dynamically

In this example, we would like to showchase how to use the [openrouteservice](https://openrouteservice.org) directions API and to avoid a road block while routing.

In [20]:
import json
import requests
import folium
import pyproj
import json

from shapely import geometry
from shapely.geometry import Point, LineString, Polygon, MultiPolygon

from openrouteservice import client
from IPython.display import *

##  Mzimuni ward, potential inundation areas

The [Dar Ramani Huria](http://ramanihuria.org) project has mapped several neighbourhoods (wards) in Dar Es Salaam to create maps of the most flood-prone areas of the city. The collected data are accessible on OpenStreetMap. With the example of the [Mzimuni ward](http://ramanihuria.org/focus-wards/mzimuni/), which is prone to being inundated during heavy rainfall, we would like to showcase the functionalities of the openrouteservie [directions API](https://openrouteservice.org/documentation/#reference/directions/directions/directions-service). Let's assume that in Mzimuni a road is blocked and we would to avoid this area by a 450 m buffer.

First let's define a function which does the buffering job with projected coordinates:

In [21]:
def CreateBufferPolygon(point_in, resolution, radius):    

    sr_wgs = pyproj.Proj(init='epsg:4326')
    sr_utm = pyproj.Proj(init='epsg:32737') # WGS84 UTM37S
    point_in_proj = pyproj.transform(sr_wgs, sr_utm, *point_in) # unpack list to arguments
    point_buffer_proj = Point(point_in_proj).buffer(radius, resolution=resolution) # 10 m buffer
    
    # Iterate over all points in buffer and build polygon
    poly_wgs = []
    for point in point_buffer_proj.exterior.coords:
        poly_wgs.append(pyproj.transform(sr_utm, sr_wgs, *point)) # Transform back to WGS84

    return poly_wgs

In [22]:
def style_function(color,opacity):
    return lambda feature: dict(color=color, weight=3, opacity=opacity)

map_params = {'tiles':'OpenStreetMap',
              'location':([-6.813522, 39.25454]),
              'zoom_start': 14}

m = folium.Map(**map_params)

# assume a road block
block_coords = [-6.813522, 39.25454]
block_poly_coords = CreateBufferPolygon(block_coords,
                                       resolution=4, 
                                       radius=450)
# appropriate start and end points
start_coords = [-6.801972, 39.246301]
end_coords = [-6.826142, 39.26561]

folium.Marker(end_coords,
              icon=folium.Icon(color='green',icon='map-marker'),
              popup='<i>start</i>').add_to(m)

folium.Marker(start_coords,
              icon=folium.Icon(color='red',icon='map-marker'),
              popup='<i>start</i>').add_to(m)

folium.Marker(damage_coords,
              icon=folium.Icon(color='orange',icon='flash'),
              popup='<i>start</i>').add_to(m)

folium.Polygon(locations=damage_poly_coords,
               color='orange',
               fill_color='orange',
               opacity=0.7,
               weight=3).add_to(m)

display(HTML('<h3>Dar Es Salaam, Mzimuni</h3>'))
display(m)

## directions API: request normal route

Ok, now let's request a normal route without considering the road block.

In [17]:
api_key = '58d904a497c67e00015b45fc40d3503b3a9a4695936156d392dbf0e3' # Individual api key
clnt = client.Client(key=api_key) # Create client with api key

# Request normal route between appropriate locations without road block
request_params = {'coordinates': [[start_coords[1], start_coords[0]],
                                 [end_coords[1], end_coords[0]]],
                'format_out': 'geojson',
                'profile': 'driving-car',
                'preference': 'shortest',
                'instructions': 'false',}

route_normal = clnt.directions(**request_params)

folium.features.GeoJson(data=route_normal,
                       name='Route without inundation ',
                        style_function=style_function('red',opacity=1),
                        overlay=True).add_to(m)

display(HTML('<h3>Dar Es Salaam, Mzimuni: Normal Route</h3>'))
display(m)

## directions API: avoid road block

Now let's request a route using avoid_polygons, which conveniently takes a GeoJSON as input.

In [12]:
damage_poly_lonlat=[(t[1],t[0]) for t in damage_poly_coords] 

request_params['options'] = {'avoid_polygons': geometry.mapping(Polygon((damage_poly_lonlat)))}

route_detour = clnt.directions(**request_params)

folium.features.GeoJson(data=route_detour,
                        name='Route with construction sites',
                        style_function=style_function('green',opacity=1),
                        overlay=True).add_to(m)

m.add_child(folium.map.LayerControl())

display(HTML('<h3>Dar Es Salaam, Mzimuni: Avoid Road Block</h3>'))
display(m)